# The Objective
This exercise revolves around translating a given SQL query into a Python script. The query in question is the ams.rules_input.sql query. 

**The objective is to generate an exact copy, in a pandas DataFrame, of data_engineer.ams_cms_input_precursor, which is a precursor to the ams.cms_input table in Python.**

## Context

### What is the Business Value of this Toy Query?
The ams.cms_input.sql query aggregates location and status data on our devices. This is required because we have different types of devices that run on different systems - our having multiple systems was an outgrowth of acquisitions (we bought a company called AccentHealth) and the organic creation of new devices (like our WiFi product, which runs on the "airtight" system). "CMS" stands for ContextMedia Systems, which is how we refer to our systems in their entirety.

### Important Notes

- **Our objective here is to reproduce the data being inserted into AMS.CMS_INPUT_HISTORY in the second statement below.**

- **The source code is production code - for this exercise, subsets of the tables have been created within the data_engineer schema in the interview database.** 

### The Query Code
The code you will be "translating" from SQL to Python is below. You may want to paste it somewhere else for easy reading, since the query is fairly long.

```sql
WITH AMS_AIRTIGHT_SOURCE_SYSTEM_NAMES AS (
    SELECT DISTINCT A.SOURCE,
                    'at'|| DENSE_RANK() OVER (ORDER BY SOURCE ASC) AS SOURCE_SYSTEM_NAME
    FROM data_engineer.airtight_sensors_history A
    WHERE A.EXPORT_DATE = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')
)

SELECT DISTINCT COALESCE(c.asset_tag, d.asset_tag, a.asset_tag) AS asset_tag,
       a.source_system,
       a.source_system_id,
       a.source_system_asset_name,
       COALESCE(c.product, d.product, e.product, f.product, g.product, a.product) AS product,
       CASE WHEN a.product_use IN ('Infusion Room Tablet','Rheumatology IRT') AND COALESCE(c.product, d.product, e.product, f.product, g.product, a.product) = 'Tablet' THEN a.product_use ELSE COALESCE(c.product, d.product, e.product, f.product, g.product, a.product) END AS product_use,
       a.cmh_id,
       a.device_location_status,
       a.last_pinged_at,
       COALESCE(c.wifi_mac_address,d.wifi_mac_address,d.lan_mac_address,e.wifi_mac_address,e.lan_mac_address,a.mac_address) AS mac_address,
       a.ssid,
       a.public_ip_address,
       COALESCE(c.sku, d.sku, e.sku, f.sku, g.sku, a.sku) AS sku,
       a.created_at,
       a.installed_date,
       a.deleted_at,
       a.reason_for_deletion,
       a.export_date
FROM (
    SELECT
            TRIM(UPPER(CASE WHEN d.asset_tag IS NOT NULL AND a.asset_id ~ '^A[0-9]+$' THEN a.asset_id --If the airtight name is good then use it
                    ELSE COALESCE(d.asset_tag,a.asset_id) --Else use the salesforce/airtight name. If not airtight use CMS name
                   END)) AS asset_tag
        ,   CASE WHEN cms_column = 'client_id' THEN 'mdm'
                 WHEN cms_column = 'radio_macaddress' THEN d.source_system_name
                 WHEN cms_column = 'display_unit_id' THEN 'broadsign'
            END AS source_system
        ,   CASE WHEN cms_column = 'client_id' THEN b.device_id
                 WHEN cms_column = 'radio_macaddress' THEN d.boxid
                 WHEN cms_column = 'display_unit_id' THEN c.host_id
            END AS source_system_id
        ,   a.asset_name AS source_system_asset_name
        ,   CASE  WHEN a.type IN ('Infusion Room Tablet','Rheumatology IRT') THEN 'Tablet'
                  ELSE a.type
            END AS product
        ,   a.type AS product_use
        ,   a.cmh_id
        ,   CASE WHEN a.status = 'Active' THEN 'Installed'
                 WHEN b.deleted_at IS NOT NULL THEN 'Deleted'
                 WHEN b.status = 'Demo' THEN 'Demo'
                 ELSE NULL::varchar
            END AS device_location_status
        ,   a.last_pinged_at
        ,   REPLACE(TRIM(UPPER(COALESCE(COALESCE(b.mac_address, b.missing_mac_address),c.mac_address,d.radio_macaddress))),':','') AS mac_address
        ,   b.ssid
        ,   c.public_ip AS public_ip_address
        ,   COALESCE(b.sku,c.sku,d.sku) AS sku
        ,   b.created_at
        ,   b.installed_date
        ,   b.deleted_at
        ,   CASE WHEN b.deleted_at IS NOT NULL THEN 'Deleted by membership' END::varchar(250) AS reason_for_deletion
        ,   a.export_date

    FROM data_engineer.shared_assets_history a

    LEFT JOIN (
                SELECT
                        a.*
                    ,   COALESCE(c.sku, b.sku) AS sku
                    ,   d.mac_address as missing_mac_address

                FROM data_engineer.mdm_devices_history a

                LEFT JOIN
                (
                 SELECT DISTINCT UPPER(a.asset_tag) AS ASSET_TAG,
                                 FIRST_VALUE(a.sku) OVER(PARTITION BY a.asset_tag ORDER BY a.created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                   FROM data_engineer.ams_sf_assets_input a
                ) b ON UPPER(a.asset_id) = b.asset_tag

                LEFT JOIN data_engineer.shared_mdm_custom_rom_sku_mapping c
                ON a.custom_rom_version = c.custom_rom_version

                LEFT JOIN data_engineer.mdm_missing_mac_addresses d    -- Some duplicate records for assets in MDM with one of which not having a MAC address, make sure that gets assigned so can be flagged as dupe
                ON a.asset_id = d.asset_id

                WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')
              ) b
         ON a.cms_id = b.client_id
        AND a.export_date = b.export_date
        AND a.cms_column = 'client_id'

    LEFT JOIN (
                SELECT
                        a.name
                    ,   a.display_unit_id
                    ,   b.poll_last_utc AS last_pinged_at
                    ,   a.host_id
                    ,   COALESCE(a.primary_mac_address,a.secondary_mac_address) AS mac_address
                    ,   b.public_ip
                    ,   COALESCE(NULLIF(c.sku,'Player'), CASE WHEN a.secondary_mac_address IS NOT NULL THEN 'P-PLA-102-NWA-01' ELSE 'P-PLA-101-NWA-01' END) AS sku --At the time of coding this was the only way to ascertain SKU from broadsign
                    ,   a.export_date

                FROM data_engineer.broadsign_hosts_history a

                LEFT JOIN data_engineer.broadsign_monitor_polls_history b
                ON a.host_id = b.client_resource_id
                AND a.export_date = b.export_date

                LEFT JOIN
                (
                 SELECT DISTINCT TRIM(UPPER(REPLACE(mac_address,':',''))) AS mac_address,
                                 FIRST_VALUE(a.sku) OVER(PARTITION BY mac_address ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                   FROM data_engineer.ams_sf_assets_input a
                ) c ON TRIM(UPPER(REPLACE(COALESCE(a.primary_mac_address,a.secondary_mac_address),':',''))) = c.mac_address

                LEFT JOIN data_engineer.ams_broadsign_migration_devices_history d
                ON a.host_id = cast(d.bs_host_id as bigint)
                AND a.export_date = d.export_date

                WHERE d.bs_host_id IS NULL
                  AND a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')
             ) c --It was a mistake to make the cms_id for broadsign display_unit_it but now I need to live with it. Therefore the below complex join is required. Once we move to ods.devices most of these joins after the FROM will not be necessary
        ON a.asset_name = c.name
        AND a.cms_id = cast(c.display_unit_id as varchar)
        AND a.export_date = c.export_date
        AND COALESCE(a.last_pinged_at,'1900-01-01') = COALESCE(c.last_pinged_at,'1900-01-01')
        AND a.cms_column = 'display_unit_id'

    LEFT JOIN (
              SELECT DISTINCT
                        FIRST_VALUE(a.boxid) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC ROWS UNBOUNDED PRECEDING) AS boxid
                    ,   FIRST_VALUE(b.source_system_name) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS source_system_name
                    ,   a.export_date
                    ,   FIRST_VALUE(a.radio_macaddress) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS radio_macaddress
                    ,   FIRST_VALUE(c.asset_tag) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS asset_tag
                    ,   FIRST_VALUE(COALESCE(c.sku,'P-WFI-101-MOJ-01')) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS sku

                FROM data_engineer.airtight_sensors_history a

                INNER JOIN ams_airtight_source_system_names b
                ON a.source = b.source

                LEFT JOIN
                (
                    SELECT DISTINCT TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) AS mac_address,
                                    FIRST_VALUE(a.asset_tag) OVER(PARTITION BY TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as asset_tag,
                                    FIRST_VALUE(a.sku) OVER(PARTITION BY TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                      FROM data_engineer.ams_sf_assets_input a
                ) c
                ON TRIM(UPPER(REPLACE(a.radio_macaddress,':',''))) = c.mac_address

                WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')
                ) d
    ON a.cms_id = d.radio_macaddress
    AND a.export_date = d.export_date
    AND a.cms_column = 'radio_macaddress'

    WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')
    ) a

LEFT JOIN data_engineer.ams_cms_input_history b --Ensure this is unique by source_system, source_system_id, export_date
ON a.source_system = b.source_system
AND a.source_system_id = b.source_system_id
AND a.export_date = b.export_date

LEFT JOIN data_engineer.ams_bat_input c
ON a.mac_address = c.wifi_mac_address

LEFT JOIN data_engineer.ams_bat_input d
ON a.mac_address = d.lan_mac_address

LEFT JOIN data_engineer.ams_bat_input e
ON a.asset_tag = e.asset_tag
AND a.product = e.product
AND a.mac_address IS NULL

LEFT JOIN data_engineer.shared_sku_master f
ON a.sku = f.sku

LEFT JOIN data_engineer.shared_sku_master g
ON a.sku = g.old_sku
AND g.rollup_sku_to_use

WHERE b.source_system_id IS NULL
  and a.source_system_asset_name NOT IN ('\nT17105') -- custom addition for the exercise
```

### Input Tables
#### Airtight Tables: WiFi Router Product Tables
- **data_engineer.airtight_sensors_history:** A table containing device location and status data on our WiFi router product. Provides hourly snapshots.

#### AMS Tables: Pre-Created Tables Used Specifically for AMS
- **data_engineer.ams_bat_input:** - A “seed” table containing a list of devices and device information. Stands for “Big Ass Table”.
- **data_engineer.ams_broadsign_migration_devices_history:** Currently an empty table awaiting data for a migration - we’re converting all our Waiting Room TV products to Broadsign (a small subsegment of them were not on Broadsign previously).
- **data_engineer.ams_cms_input_history:** A table containing a historical record of daily snapshots of ams.cms_input.
- **data_engineer.ams_rules_input:** A table that contains various business parameters business users can change. In the absence of named variables (and because AMS was built from an analyst's point of view, where SQL and Redshift were the only infrastructure possibilities), this is how parameters are created for AMS.
- **data_engineer.ams_sf_assets_input:** A generated input table that contains any device information contained in Salesforce. This is primarily for obtaining data on our non-interactive devices, like our TVs.

#### Broadsign Tables: Waiting Room TV Product Tables
- **data_engineer.broadsign_hosts_history:** Device information on our Waiting Room TV products (the majority of which run on Broadsign - a content delivery platform). 
- **data_engineer.broadsign_monitor_polls_history:** Ping information on our Waiting Room TV products.

#### Mobile Device Management (MDM) Tables: Tables for Device Data on All Other Devices (Tablets and Wallboards)
- **data_engineer.mdm_devices_history:** A historical record of information on our devices - MAC address, sku, hardware info, etc.
- **data_engineer.mdm_missing_mac_addresses:** A custom table created to identify the MAC address for devices were missing them.

#### Shared Tables: Pre-Created Tables in Redshift Used for Multiple Processes, Including AMS
- **data_engineer.shared_assets_history:** A legacy table of old asset, product, and status information. We primarily use this as a “seed table” for AMS, which generates a newer version of asset, location, and status information.
- **data_engineer.shared_mdm_custom_rom_sku_mapping:** A custom mapping table that helps us determine skus appropriately.
- **data_engineer.shared_sku_master:** - Another “seed” table containing manually entered information on sku matching (this was needed to match old skus to newer ones).

### Output Tables
- **data_engineer.ams_cms_input_precursor:** A precursor to ams.cms_input, which is a cleaned up, aggregated table containing location and status information on our devices - meant for AMS consumption. For our intents and purposes, this is what gets stored in the ams.cms_input_history table as a snapshot.

## Pulling the Data
### Accessing the Input Data
For speed and optimization reasons, i'm only pulling the required data. For doing so, i went through the SQL query and identified which columns of which tables were needed, and also if there were any "WHERE" conditions which would help me reduce the final amount of data that was being downloaded.

In [9]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
from datetime import datetime

pd.options.display.max_colwidth = 200 

#Creating connection to the database
conn = psycopg2.connect(host="data-interview.outcomehealth.io",
                        port="5432",
                        database="product_analytics", 
                        user="pa_candidate", 
                        password="OsOntUnDleYeTivi")

# Getting all the required data into pandas dataframes
query_airtight_sensors_history="SELECT boxid,radio_macaddress,export_date,export_hour,radio_upsince,source FROM data_engineer.airtight_sensors_history WHERE EXPORT_DATE = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"
airtight_sensors_history = sqlio.read_sql_query(query_airtight_sensors_history, conn)

query_ams_rules_input='SELECT value,rule FROM data_engineer.ams_rules_input'
ams_rules_input = sqlio.read_sql_query(query_ams_rules_input,conn)

query_ams_bat_input='SELECT wifi_mac_address,lan_mac_address,asset_tag,product,sku FROM data_engineer.ams_bat_input'
ams_bat_input = sqlio.read_sql_query(query_ams_bat_input,conn)

query_shared_sku_master='SELECT sku,old_sku,rollup_sku_to_use,product FROM data_engineer.shared_sku_master' 
shared_sku_master = sqlio.read_sql_query(query_shared_sku_master,conn)

query_shared_assets_history="SELECT asset_id,asset_name,type,status,cmh_id,last_pinged_at,export_date,cms_id,cms_column FROM data_engineer.shared_assets_history WHERE export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date') AND asset_name NOT IN ('\nT17105')"
shared_assets_history = sqlio.read_sql_query(query_shared_assets_history,conn)

query_mdm_devices_history="SELECT id,asset_id,custom_rom_version,device_id,client_id,status,mac_address,ssid,created_at,installed_date,deleted_at,export_date FROM data_engineer.mdm_devices_history WHERE export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"
mdm_devices_history = sqlio.read_sql_query(query_mdm_devices_history,conn)

query_ams_sf_assets_input='SELECT asset_tag,sku,created_date,serial_number,mac_address FROM data_engineer.ams_sf_assets_input'
ams_sf_assets_input = sqlio.read_sql_query(query_ams_sf_assets_input,conn)

query_shared_mdm_custom_rom_sku_mapping='SELECT custom_rom_version,sku FROM data_engineer.shared_mdm_custom_rom_sku_mapping'
shared_mdm_custom_rom_sku_mapping = sqlio.read_sql_query(query_shared_mdm_custom_rom_sku_mapping,conn)

query_mdm_missing_mac_addresses='SELECT asset_id,mac_address FROM data_engineer.mdm_missing_mac_addresses'
mdm_missing_mac_addresses = sqlio.read_sql_query(query_mdm_missing_mac_addresses,conn)

query_broadsign_hosts_history="SELECT name,display_unit_id,host_id,primary_mac_address,secondary_mac_address,export_date FROM data_engineer.broadsign_hosts_history WHERE export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"
broadsign_hosts_history = sqlio.read_sql_query(query_broadsign_hosts_history,conn)

query_broadsign_monitor_polls_history='SELECT poll_last_utc,public_ip,client_resource_id,export_date FROM data_engineer.broadsign_monitor_polls_history'
broadsign_monitor_polls_history = sqlio.read_sql_query(query_broadsign_monitor_polls_history,conn)

query_ams_broadsign_migration_devices_history='SELECT bs_host_id,export_date FROM data_engineer.ams_broadsign_migration_devices_history WHERE bs_host_id IS NULL'
ams_broadsign_migration_devices_history = sqlio.read_sql_query(query_ams_broadsign_migration_devices_history,conn)

query_ams_cms_input_history='SELECT source_system,source_system_id,export_date FROM data_engineer.ams_cms_input_history WHERE source_system_id IS NULL'
ams_cms_input_history = sqlio.read_sql_query(query_ams_cms_input_history,conn)


### Accessing the Output Data
For accessing the output dataset, i just run the whole query replacing the **ams_airtight_source_system_names** table into the query for it's definition (inner join line 132) . This will give me the the data i need to compare my results later on, once i have done everything that is required to solve this problem.

In [2]:
sql = """SELECT DISTINCT COALESCE(c.asset_tag, d.asset_tag, a.asset_tag) AS asset_tag,
       a.source_system,
       a.source_system_id,
       a.source_system_asset_name,
       COALESCE(c.product, d.product, e.product, f.product, g.product, a.product) AS product,
       CASE WHEN a.product_use IN ('Infusion Room Tablet','Rheumatology IRT') AND COALESCE(c.product, d.product, e.product, f.product, g.product, a.product) = 'Tablet' THEN a.product_use ELSE COALESCE(c.product, d.product, e.product, f.product, g.product, a.product) END AS product_use,
       a.cmh_id,
       a.device_location_status,
       a.last_pinged_at,
       COALESCE(c.wifi_mac_address,d.wifi_mac_address,d.lan_mac_address,e.wifi_mac_address,e.lan_mac_address,a.mac_address) AS mac_address,
       a.ssid,
       a.public_ip_address,
       COALESCE(c.sku, d.sku, e.sku, f.sku, g.sku, a.sku) AS sku,
       a.created_at,
       a.installed_date,
       a.deleted_at,
       a.reason_for_deletion,
       a.export_date
FROM (
    SELECT
            TRIM(UPPER(CASE WHEN d.asset_tag IS NOT NULL AND a.asset_id ~ '^A[0-9]+$' THEN a.asset_id --If the airtight name is good then use it
                    ELSE COALESCE(d.asset_tag,a.asset_id) --Else use the salesforce/airtight name. If not airtight use CMS name
                   END)) AS asset_tag
        ,   CASE WHEN cms_column = 'client_id' THEN 'mdm'
                 WHEN cms_column = 'radio_macaddress' THEN d.source_system_name
                 WHEN cms_column = 'display_unit_id' THEN 'broadsign'
            END AS source_system
        ,   CASE WHEN cms_column = 'client_id' THEN b.device_id
                 WHEN cms_column = 'radio_macaddress' THEN d.boxid
                 WHEN cms_column = 'display_unit_id' THEN c.host_id
            END AS source_system_id
        ,   a.asset_name AS source_system_asset_name
        ,   CASE  WHEN a.type IN ('Infusion Room Tablet','Rheumatology IRT') THEN 'Tablet'
                  ELSE a.type
            END AS product
        ,   a.type AS product_use
        ,   a.cmh_id
        ,   CASE WHEN a.status = 'Active' THEN 'Installed'
                 WHEN b.deleted_at IS NOT NULL THEN 'Deleted'
                 WHEN b.status = 'Demo' THEN 'Demo'
                 ELSE NULL::varchar
            END AS device_location_status
        ,   a.last_pinged_at
        ,   REPLACE(TRIM(UPPER(COALESCE(COALESCE(b.mac_address, b.missing_mac_address),c.mac_address,d.radio_macaddress))),':','') AS mac_address
        ,   b.ssid
        ,   c.public_ip AS public_ip_address
        ,   COALESCE(b.sku,c.sku,d.sku) AS sku
        ,   b.created_at
        ,   b.installed_date
        ,   b.deleted_at
        ,   CASE WHEN b.deleted_at IS NOT NULL THEN 'Deleted by membership' END::varchar(250) AS reason_for_deletion
        ,   a.export_date

    FROM data_engineer.shared_assets_history a

        LEFT JOIN (
                    SELECT
                            a.*
                        ,   COALESCE(c.sku, b.sku) AS sku
                        ,   d.mac_address as missing_mac_address

                    FROM data_engineer.mdm_devices_history a

                        LEFT JOIN (
                          
                            SELECT DISTINCT UPPER(a.asset_tag) AS ASSET_TAG,
                                            FIRST_VALUE(a.sku) OVER(PARTITION BY a.asset_tag ORDER BY a.created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                            FROM data_engineer.ams_sf_assets_input a
                        ) b 
                        ON UPPER(a.asset_id) = b.asset_tag

                        LEFT JOIN data_engineer.shared_mdm_custom_rom_sku_mapping c
                        ON a.custom_rom_version = c.custom_rom_version

                        LEFT JOIN data_engineer.mdm_missing_mac_addresses d    -- Some duplicate records for assets in MDM with one of which not having a MAC address, make sure that gets assigned so can be flagged as dupe
                        ON a.asset_id = d.asset_id

                    WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')
                  ) b
            ON a.cms_id = b.client_id
            AND a.export_date = b.export_date
            AND a.cms_column = 'client_id'

        LEFT JOIN (
                    SELECT
                            a.name
                        ,   a.display_unit_id
                        ,   b.poll_last_utc AS last_pinged_at
                        ,   a.host_id
                        ,   COALESCE(a.primary_mac_address,a.secondary_mac_address) AS mac_address
                        ,   b.public_ip
                        ,   COALESCE(NULLIF(c.sku,'Player'), CASE WHEN a.secondary_mac_address IS NOT NULL THEN 'P-PLA-102-NWA-01' ELSE 'P-PLA-101-NWA-01' END) AS sku --At the time of coding this was the only way to ascertain SKU from broadsign
                        ,   a.export_date

                    FROM data_engineer.broadsign_hosts_history a

                        LEFT JOIN data_engineer.broadsign_monitor_polls_history b
                        ON a.host_id = b.client_resource_id
                        AND a.export_date = b.export_date

                        LEFT JOIN
                        (
                         SELECT DISTINCT TRIM(UPPER(REPLACE(mac_address,':',''))) AS mac_address,
                                         FIRST_VALUE(a.sku) OVER(PARTITION BY mac_address ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                           FROM data_engineer.ams_sf_assets_input a
                        ) c 
                        ON TRIM(UPPER(REPLACE(COALESCE(a.primary_mac_address,a.secondary_mac_address),':',''))) = c.mac_address

                        LEFT JOIN data_engineer.ams_broadsign_migration_devices_history d
                        ON a.host_id = cast(d.bs_host_id as bigint)
                        AND a.export_date = d.export_date

                    WHERE d.bs_host_id IS NULL AND a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')
                 ) c --It was a mistake to make the cms_id for broadsign display_unit_it but now I need to live with it. Therefore the below complex join is required. Once we move to ods.devices most of these joins after the FROM will not be necessary
            ON a.asset_name = c.name
            AND a.cms_id = cast(c.display_unit_id as varchar)
            AND a.export_date = c.export_date
            AND COALESCE(a.last_pinged_at,'1900-01-01') = COALESCE(c.last_pinged_at,'1900-01-01')
            AND a.cms_column = 'display_unit_id'

        LEFT JOIN (
                SELECT DISTINCT
                            FIRST_VALUE(a.boxid) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC ROWS UNBOUNDED PRECEDING) AS boxid
                        ,   FIRST_VALUE(b.source_system_name) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS source_system_name
                        ,   a.export_date
                        ,   FIRST_VALUE(a.radio_macaddress) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS radio_macaddress
                        ,   FIRST_VALUE(c.asset_tag) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS asset_tag
                        ,   FIRST_VALUE(COALESCE(c.sku,'P-WFI-101-MOJ-01')) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS sku

                FROM data_engineer.airtight_sensors_history a

                    INNER JOIN(
                    			SELECT DISTINCT A.SOURCE,
                    							'at'|| DENSE_RANK() OVER (ORDER BY SOURCE ASC) AS SOURCE_SYSTEM_NAME
    							FROM data_engineer.airtight_sensors_history A
    							WHERE A.EXPORT_DATE = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')) b
                    ON a.source = b.source

                    LEFT JOIN(
                          SELECT DISTINCT TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) AS mac_address,
                                        FIRST_VALUE(a.asset_tag) OVER(PARTITION BY TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as asset_tag,
                                        FIRST_VALUE(a.sku) OVER(PARTITION BY TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                          FROM data_engineer.ams_sf_assets_input a
                            ) c
                    ON TRIM(UPPER(REPLACE(a.radio_macaddress,':',''))) = c.mac_address

                WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')

                   ) d 
            ON a.cms_id = d.radio_macaddress
            AND a.export_date = d.export_date
            AND a.cms_column = 'radio_macaddress'

    WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')
    ) a

    LEFT JOIN data_engineer.ams_cms_input_history b --Ensure this is unique by source_system, source_system_id, export_date
    ON a.source_system = b.source_system
    AND a.source_system_id = b.source_system_id
    AND a.export_date = b.export_date

    LEFT JOIN data_engineer.ams_bat_input c
    ON a.mac_address = c.wifi_mac_address

    LEFT JOIN data_engineer.ams_bat_input d
    ON a.mac_address = d.lan_mac_address

    LEFT JOIN data_engineer.ams_bat_input e
    ON a.asset_tag = e.asset_tag
    AND a.product = e.product
    AND a.mac_address IS NULL

    LEFT JOIN data_engineer.shared_sku_master f
    ON a.sku = f.sku

    LEFT JOIN data_engineer.shared_sku_master g
    ON a.sku = g.old_sku
    AND g.rollup_sku_to_use

WHERE b.source_system_id IS NULL and a.source_system_asset_name NOT IN ('\nT17105')"""
output_data = sqlio.read_sql_query(sql, conn)
output_data.head()

,asset_tag,source_system,source_system_id,source_system_asset_name,product,product_use,cmh_id,device_location_status,last_pinged_at,mac_address,ssid,public_ip_address,sku,created_at,installed_date,deleted_at,reason_for_deletion,export_date
0,0505,broadsign,NaN,0505 - Aopen Retired 10/28/13,Waiting Room Screen,Waiting Room Screen,NaN,None,2013-05-01 16:43:38,None,None,None,None,NaT,None,NaT,None,2019-03-31
1,0516,broadsign,NaN,0516-1 (WIll be MIA often) - lost/stolen,Waiting Room Screen,Waiting Room Screen,NaN,None,2008-10-06 13:20:37,None,None,None,None,NaT,None,NaT,None,2019-03-31
2,10005,broadsign,NaN,0501-1_10005,Waiting Room Screen,Waiting Room Screen,501.0,None,2008-06-26 19:44:16,00018068E25D,None,None,P-PLA-101-NWA-01,NaT,None,NaT,None,2019-03-31
3,10016,broadsign,NaN,5056-2_10016 - MS /66856/ - Returning 03/18/2016,Waiting Room Screen,Waiting Room Screen,5056.0,None,2015-11-20 02:30:47,None,None,None,None,NaT,None,NaT,None,2019-03-31
4,10021,broadsign,NaN,30180-1_10021 - Combo,Waiting Room Screen,Waiting Room Screen,30180.0,None,2015-04-17 15:00:22,None,None,None,None,NaT,None,NaT,None,2019-03-31


## Addressing the problem

The first thing i did to solve this problem was to take some time to break the problem down into smaller pieces. After doing so, i recognized 3 small tables that needed to be created before anything else. These tables are **"b*"**,**"c*"** and **"d*"** (which all together they compose **"a"**). Also, there is one table called **"ams_airtight_source_system_names"**, which is the very first piece of code that appears in the SQL code, which is used for an inner join later on in the query, the one that generates table **"d*"**.

As i said, **"b*"**,**"c*"** and **"d*"** compose **"a"**, and once i have **"a"**, all i have to do is some left joins ( with  **"b"**, **"c"**, **"d"**, **"e"**, **"f"** and **"g"**) to get the final table from which i will take the final data (the first SELECT DISTINCT that one runs into when reading the query), to emulate the result of the SQL query with a pandas dataframe. 

As a good practice, i prefer validating the results on the go, so every step i take i check if the result i have is the one that im supposed to.



## Processing the Data

### First things first, getting **"ams_airtight_source_system_names":**


In [10]:
#Creating an empty panda dataframe
ams_airtight_source_system_names = pd.DataFrame()

#Filling in the dataframe with the required data

#Creating the 'source' column and filling it with unique values
ams_airtight_source_system_names['source'] = airtight_sensors_history['source'].drop_duplicates()

#Creating a rank column and passing the returned rank series, concatenating a string 'at' to each row
ams_airtight_source_system_names['source_system_name'] ='at'+airtight_sensors_history.source.rank(method='dense').astype('int').astype('str')

#Reseting the index and deleting the old one (drop=True)
ams_airtight_source_system_names.reset_index(drop=True,inplace=True)

### Validating result

In [11]:
#SQL query which returns the 'ams_airtight_source_system_names' table
sql = """SELECT DISTINCT A.SOURCE,
                    'at'|| DENSE_RANK() OVER (ORDER BY SOURCE ASC) AS SOURCE_SYSTEM_NAME
    FROM data_engineer.airtight_sensors_history A
    WHERE A.EXPORT_DATE = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"""

#Fetching the resulting data from the sql query
data = sqlio.read_sql_query(sql, conn)

# I check if both tables have the same length, and if so i merge them and then locate the rows where the column
# "_merge" is different from 'both', in other words, if both tables are equal , the dataframe 'foo' will be
# empty
if len(data) == len(ams_airtight_source_system_names):
    foo = data.merge(ams_airtight_source_system_names,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
    if len(foo) == 0:
        print("Both tables are equal!")
    else:
        print("Both tables are not equal")
else:
    print("The lenght of each table is different")

Both tables are equal!


### Getting b*:

In [ ]:
b= """SELECT
                            a.*
                        ,   COALESCE(c.sku, b.sku) AS sku
                        ,   d.mac_address as missing_mac_address

                    FROM data_engineer.mdm_devices_history a

                        LEFT JOIN (
                          
                            SELECT DISTINCT UPPER(a.asset_tag) AS ASSET_TAG,
                                            FIRST_VALUE(a.sku) OVER(PARTITION BY a.asset_tag ORDER BY a.created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                            FROM data_engineer.ams_sf_assets_input a
                        ) b 
                        ON UPPER(a.asset_id) = b.asset_tag

                        LEFT JOIN data_engineer.shared_mdm_custom_rom_sku_mapping c
                        ON a.custom_rom_version = c.custom_rom_version

                        LEFT JOIN data_engineer.mdm_missing_mac_addresses d    -- Some duplicate records for assets in MDM with one of which not having a MAC address, make sure that gets assigned so can be flagged as dupe
                        ON a.asset_id = d.asset_id

                    WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"""

In [12]:
# Parsing the data in column 'created_date' into datetime.date objects, since i dont know in which data type
# the data is, i use try/except 

try:
	ams_sf_assets_input['created_date'] = ams_sf_assets_input['created_date'].apply(lambda x: datetime.date(x))
except Exception as e:
    # Data already in datetime.date type
	if str(e) == "descriptor 'date' requires a 'datetime.datetime' object but received a 'datetime.date'":
		pass
	else:
		print(str(e))

#Sorting the dataframe by 'created_date' in descending order
ams_sf_assets_input.sort_values(['created_date'],ascending=False,inplace=True)

#Filling the NaNs values in 'asset_tag' column with a string 'None' so that the group by doesn't filter 
#them out
ams_sf_assets_input['asset_tag'].fillna('None',inplace=True)

#Grouping the dataframe by 'asset_tag' and selecting the first 'sku' value per window 
b=ams_sf_assets_input.groupby('asset_tag').agg({'sku':'first'}).reset_index()

#Parsing the column to upper case (SELECT DISTINCT UPPER(a.asset_tag) AS ASSET_TAG...)
b['asset_tag']=b['asset_tag'].str.upper() 

#Since the first LEFT JOIN uses 'asset_id' from the 'mdm_device_history' as the condition for the join, and
#since b uses 'asset_tag', which has already been filled in with 'None' for the NaN values, im parsing the 
#'asset_id' as well
mdm_devices_history['asset_id'].fillna('None',inplace=True)


#Getting the 'asset_tag' column to upper case for the "ON" condition in the first left join, this wont have
#any effects going forward since im not using that column outside this first big "SELECT" that compose b*
mdm_devices_history['asset_id']=mdm_devices_history['asset_id'].str.upper()

#Same reason as above
mdm_missing_mac_addresses['asset_id']=mdm_missing_mac_addresses['asset_id'].str.upper()

#LEFT JOINS

#First left join, im using suffixes in case both tables have any columns with the same name
s1 = pd.merge(mdm_devices_history, b, how='left', left_on= ['asset_id'],right_on=['asset_tag'],suffixes=('_a','_b'))


s2 = pd.merge(s1, shared_mdm_custom_rom_sku_mapping, how='left', left_on=['custom_rom_version'],right_on=['custom_rom_version'],suffixes=('','_c'))


s3 = pd.merge(s2, mdm_missing_mac_addresses, how='left', left_on=['asset_id'],right_on=['asset_id'],suffixes=('','_d'))

#Finally getting b* by concatenating s3 columns, since i have a*, which means every column from a, i use 
#list('mdm_devices_history'). The 'combine_first' emulates the  SQL function 'COALESCE', which returns the 
#first non-null value from sku_c and sku in this case 
b=pd.concat([s3[list(mdm_devices_history.columns)],s3.sku_c.combine_first(s3.sku)],axis=1)

#Changing names
b['missing_mac_address']=s3['mac_address_d']

### Validating b*:

In [13]:
sql=""" SELECT
                            a.*
                        ,   COALESCE(c.sku, b.sku) AS sku
                        ,   d.mac_address as missing_mac_address

                    FROM data_engineer.mdm_devices_history a

                        LEFT JOIN (
                          
                            SELECT DISTINCT UPPER(a.asset_tag) AS ASSET_TAG,
                                            FIRST_VALUE(a.sku) OVER(PARTITION BY a.asset_tag ORDER BY a.created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                            FROM data_engineer.ams_sf_assets_input a
                        ) b 
                        ON UPPER(a.asset_id) = b.asset_tag

                        LEFT JOIN data_engineer.shared_mdm_custom_rom_sku_mapping c
                        ON a.custom_rom_version = c.custom_rom_version

                        LEFT JOIN data_engineer.mdm_missing_mac_addresses d    -- Some duplicate records for assets in MDM with one of which not having a MAC address, make sure that gets assigned so can be flagged as dupe
                        ON a.asset_id = d.asset_id

                    WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"""
data = sqlio.read_sql_query(sql, conn)

if len(data) == len(b):
    #As i explained before, given that i used upper case for the 'asset_tag' column, i need to change it 
    #as well in the 'data' dataframe, which comes from SQL. This is not gonna be a problem going forward
    #because b*['asset_id'] is not being used anywhere else
    data['asset_id']=data['asset_id'].str.upper()
    foo = data.merge(b,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
    if len(foo) == 0:
        print("Both tables are equal!")
    else:
        print("Both tables are not equal")
else:
    print("The lenght of each table is different")

Both tables are equal!


### Getting c*:

In [ ]:
c="""SELECT
                            a.name
                        ,   a.display_unit_id
                        ,   b.poll_last_utc AS last_pinged_at
                        ,   a.host_id
                        ,   COALESCE(a.primary_mac_address,a.secondary_mac_address) AS mac_address
                        ,   b.public_ip
                        ,   COALESCE(NULLIF(c.sku,'Player'), CASE WHEN a.secondary_mac_address IS NOT NULL THEN 'P-PLA-102-NWA-01' ELSE 'P-PLA-101-NWA-01' END) AS sku --At the time of coding this was the only way to ascertain SKU from broadsign
                        ,   a.export_date

                    FROM data_engineer.broadsign_hosts_history a

                        LEFT JOIN data_engineer.broadsign_monitor_polls_history b
                        ON a.host_id = b.client_resource_id
                        AND a.export_date = b.export_date

                        LEFT JOIN
                        (
                         SELECT DISTINCT TRIM(UPPER(REPLACE(mac_address,':',''))) AS mac_address,
                                         FIRST_VALUE(a.sku) OVER(PARTITION BY mac_address ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                           FROM data_engineer.ams_sf_assets_input a
                        ) c 
                        ON TRIM(UPPER(REPLACE(COALESCE(a.primary_mac_address,a.secondary_mac_address),':',''))) = c.mac_address

                        LEFT JOIN data_engineer.ams_broadsign_migration_devices_history d
                        ON a.host_id = cast(d.bs_host_id as bigint)
                        AND a.export_date = d.export_date

                    WHERE d.bs_host_id IS NULL AND a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"""

In [14]:
#Filling the NaNs values in 'mac_addres' column with a string 'None' so that the group by doesn't filter 
#them out
ams_sf_assets_input['mac_address'].fillna('None',inplace=True)

#Grouping the dataframe by 'asset_tag' and selecting the first 'sku' value per window 
c=ams_sf_assets_input.groupby('mac_address').agg({'sku':'first'}).reset_index()

#LEFT JOINS


s1 = pd.merge(broadsign_hosts_history, broadsign_monitor_polls_history, how='left', left_on=['host_id','export_date'],right_on=['client_resource_id','export_date'],suffixes=('_a','_b'))


s2 = pd.merge(s1, c, how='left', left_on=[broadsign_hosts_history.primary_mac_address.combine_first(broadsign_hosts_history.secondary_mac_address).str.replace(':','').str.upper().str.strip()],right_on=['mac_address'],suffixes=('','_c'))


s3 = pd.merge(s2, ams_broadsign_migration_devices_history, how='left', left_on=[broadsign_hosts_history['host_id'],broadsign_hosts_history['export_date']],right_on=[ams_broadsign_migration_devices_history.bs_host_id.astype('int64'),'export_date'],suffixes=('','_d'))

def case(x):
    if x == None:
        return 'P-PLA-101-NWA-01'
    else:
        return 'P-PLA-102-NWA-01'

def nullif(x):
	if x == 'Player':
		return None
	else:
		return x

#Swapping NaNs for None 
s3['secondary_mac_address'].fillna('None').replace(to_replace='None',value=None,inplace=True)

#Getting c by concatenating s3 columns
c=pd.concat([s3[['name','display_unit_id','poll_last_utc','host_id','public_ip','export_date']],s3.primary_mac_address.combine_first(s3.secondary_mac_address)],axis=1)
c = c.rename(columns={'poll_last_utc': 'last_pinged_at','primary_mac_address':'mac_address'})

s3['sku'].fillna('None').replace(to_replace='None',value=None,inplace=True)
c['sku'] = s3['sku'].apply(nullif).combine_first(s3['secondary_mac_address'].apply(case))


### Validating c*:

In [15]:
sql = """SELECT
                            a.name
                        ,   a.display_unit_id
                        ,   b.poll_last_utc AS last_pinged_at
                        ,   a.host_id
                        ,   COALESCE(a.primary_mac_address,a.secondary_mac_address) AS mac_address
                        ,   b.public_ip
                        ,   COALESCE(NULLIF(c.sku,'Player'), CASE WHEN a.secondary_mac_address IS NOT NULL THEN 'P-PLA-102-NWA-01' ELSE 'P-PLA-101-NWA-01' END) AS sku --At the time of coding this was the only way to ascertain SKU from broadsign
                        ,   a.export_date

                    FROM data_engineer.broadsign_hosts_history a

                        LEFT JOIN data_engineer.broadsign_monitor_polls_history b
                        ON a.host_id = b.client_resource_id
                        AND a.export_date = b.export_date

                        LEFT JOIN
                        (
                         SELECT DISTINCT TRIM(UPPER(REPLACE(mac_address,':',''))) AS mac_address,
                                         FIRST_VALUE(a.sku) OVER(PARTITION BY mac_address ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                           FROM data_engineer.ams_sf_assets_input a
                        ) c 
                        ON TRIM(UPPER(REPLACE(COALESCE(a.primary_mac_address,a.secondary_mac_address),':',''))) = c.mac_address

                        LEFT JOIN data_engineer.ams_broadsign_migration_devices_history d
                        ON a.host_id = cast(d.bs_host_id as bigint)
                        AND a.export_date = d.export_date

                    WHERE d.bs_host_id IS NULL AND a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"""
data = sqlio.read_sql_query(sql, conn)

if len(data) == len(c):
    foo = data.merge(c,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
    if len(foo) == 0:
        print("Both tables are equal!")
    else:
        print("Both tables are not equal")
else:
    print("The lenght of each table is different")

Both tables are equal!


### Getting d:

In [ ]:
d="""SELECT DISTINCT
                            FIRST_VALUE(a.boxid) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC ROWS UNBOUNDED PRECEDING) AS boxid
                        ,   FIRST_VALUE(b.source_system_name) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS source_system_name
                        ,   a.export_date
                        ,   FIRST_VALUE(a.radio_macaddress) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS radio_macaddress
                        ,   FIRST_VALUE(c.asset_tag) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS asset_tag
                        ,   FIRST_VALUE(COALESCE(c.sku,'P-WFI-101-MOJ-01')) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS sku

                FROM data_engineer.airtight_sensors_history a

                    INNER JOIN ams_airtight_source_system_names b
                    ON a.source = b.source

                    LEFT JOIN(
                          SELECT DISTINCT TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) AS mac_address,
                                        FIRST_VALUE(a.asset_tag) OVER(PARTITION BY TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as asset_tag,
                                        FIRST_VALUE(a.sku) OVER(PARTITION BY TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                          FROM data_engineer.ams_sf_assets_input a
                            ) c
                    ON TRIM(UPPER(REPLACE(a.radio_macaddress,':',''))) = c.mac_address

                WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"""

In [6]:
def combine_first_str(x):
    if x == 'None':
        return 'P-WFI-101-MOJ-01'
    else:
        return x

#ams_sf_assets_input['created_date'] = ams_sf_assets_input['created_date'].apply(lambda x: datetime.date(x))
#ams_sf_assets_input.sort_values(['created_date'],ascending=False,inplace=True)

ams_sf_assets_input['asset_tag'].fillna('None',inplace=True)

c=ams_sf_assets_input.groupby(ams_sf_assets_input['serial_number'].combine_first(ams_sf_assets_input['mac_address']).str.replace(':','').str.upper().str.strip().fillna('None')).agg({'asset_tag':'first','sku':'first'}).reset_index().drop_duplicates()
c=c.rename(columns={'serial_number':'mac_address'})


airtight_sensors_history['foo']=airtight_sensors_history['radio_macaddress'].str.replace(':','').str.upper().str.strip()
airtight_sensors_history['radio_macaddress'].fillna('None',inplace=True)
#airtight_sensors_history['export_date'] = airtight_sensors_history['export_date'].apply(lambda x: datetime.date(x)) ya esta en datetime
#airtight_sensors_history['radio_upsince'] = airtight_sensors_history['radio_upsince'].apply(lambda x: datetime.date(x))
#airtight_sensors_history['export_hour'] = airtight_sensors_history['export_hour'].apply(lambda x: datetime.date(x)) ya esta en datetime

s1=pd.merge(airtight_sensors_history,ams_airtight_source_system_names, on='source', how='inner')



s2=pd.merge(s1,c,how='left', right_on='mac_address', left_on='foo')

s2['sku'].fillna('None',inplace=True)
s2['sku']=s2['sku'].apply(combine_first_str)

s2.sort_values(['export_hour','radio_upsince'],ascending=[False,False])
d=s2.groupby(['radio_macaddress', 'export_date']).agg({'boxid':'first','source_system_name':'first','radio_macaddress':'first','asset_tag':'first','sku':'first'}).reset_index(drop=True)

### Validating d:

In [8]:
sql = """SELECT DISTINCT
                            FIRST_VALUE(a.boxid) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC ROWS UNBOUNDED PRECEDING) AS boxid
                        ,   FIRST_VALUE(b.source_system_name) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS source_system_name
                        ,   a.export_date
                        ,   FIRST_VALUE(a.radio_macaddress) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS radio_macaddress
                        ,   FIRST_VALUE(c.asset_tag) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS asset_tag
                        ,   FIRST_VALUE(COALESCE(c.sku,'P-WFI-101-MOJ-01')) OVER (PARTITION BY a.radio_macaddress, a.export_date ORDER BY a.export_hour DESC, a.radio_upsince DESC  ROWS UNBOUNDED PRECEDING) AS sku

                FROM data_engineer.airtight_sensors_history a

                    INNER JOIN (    SELECT DISTINCT A.SOURCE,
                    'at'|| DENSE_RANK() OVER (ORDER BY SOURCE ASC) AS SOURCE_SYSTEM_NAME
    FROM data_engineer.airtight_sensors_history A
    WHERE A.EXPORT_DATE = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')) b
                    ON a.source = b.source

                    LEFT JOIN(
                          SELECT DISTINCT TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) AS mac_address,
                                        FIRST_VALUE(a.asset_tag) OVER(PARTITION BY TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as asset_tag,
                                        FIRST_VALUE(a.sku) OVER(PARTITION BY TRIM(UPPER(REPLACE(COALESCE(serial_number, mac_address),':',''))) ORDER BY created_date DESC ROWS UNBOUNDED PRECEDING) as sku
                          FROM data_engineer.ams_sf_assets_input a
                            ) c
                    ON TRIM(UPPER(REPLACE(a.radio_macaddress,':',''))) = c.mac_address

                WHERE a.export_date = (SELECT value::date FROM data_engineer.ams_rules_input WHERE rule = 'Rollforward date')"""
data = sqlio.read_sql_query(sql, conn)

if len(data) == len(d):
    foo = data.merge(d,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
    if len(foo) == 0:
        print("Both tables are equal!")
    else:
        print("Both tables are not equal")
else:
    print("The lenght of each table is different")

Both tables are equal!


## Output Validation
For validating that every value in the created table is identical to the output table.

## Unit Test Implementation
We ask that you implement at least one unit test to catch errors with your script.